In [116]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [2]:
browser = webdriver.Chrome("./chromedriver")
browser.get("https://www.linkedin.com")

In [3]:
username = browser.find_element_by_id("session_key")
username.send_keys("mjeng@ucsc.edu")
password = browser.find_element_by_id("session_password")
password.send_keys("CSE115Project")

In [4]:
login_button=browser.find_element_by_class_name("sign-in-form__submit-button")
login_button.click()

In [119]:
browser.get("https://www.linkedin.com/jobs")

In [120]:
job_search_bar = browser.find_element_by_id("jobs-search-box-keyword-id-ember37")
job_search_bar.send_keys("Software Engineer")

In [121]:
job_search_button = browser.find_elements_by_class_name("jobs-search-box__submit-button")
job_search_button[0].click()
#Potential bug - not able to find element unless i manually inspect  - https://stackoverflow.com/questions/50698342/selenium-cant-find-elements-until-i-inspect-the-page

In [180]:
# ------------------ Scraping the Linkedin Job Webpage's left rail (it's a two pane wrapper)
 # parsing the visible webpage
pageSource = browser.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')
# browser.implicitly_wait(10)
#job_container = lxml_soup.find('ul', class_ = 'jobs-search-results__list')
job_container = lxml_soup.find_all('li', class_ = 'jobs-search-results__list-item')

# element = browser.find_element_by_id("ember665")
# actions = ActionChains(driver)
# actions.move_to_element(element).perform()
# driver.execute_script("arguments[0].scrollIntoView();", element)
# time.sleep(3)
#browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#Above doesn't seem to grab the entire html thing

#print(job_container[0])
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 25 jobs.


In [181]:
job_container[20]

<li class="jobs-search-results__list-item occludable-update p0 relative jobs-search-results__job-card-search--generic-occludable-area ember-view" data-occludable-entity-urn="urn:li:fs_normalized_jobPosting:2502454326" id="ember312"><!-- --></li>

In [182]:
import re
id_list = []
for job in job_container:
    job = str(job)
    numbers = re.findall(r'[0-9]+',job)
    for i in numbers:
        if len(i) == 10: #hardcoding that ID is always 10 digits
            id_list.append(i)
            break

print(id_list)
print(len(id_list))

['2473948505', '2486288184', '2477257225', '2473932077', '2469597134', '2505171031', '2486211179', '2484382977', '2487428041', '2483648907', '2482229352', '2487423831', '2479908835', '2486221840', '2482218586', '2504184136', '2504450588', '2504174911', '2486299333', '2505137753', '2502454326', '2487425343', '2483653356', '2504833347', '2482860227']
25


In [187]:
job_links = []
for id in id_list: 
    currUrl = driver.current_url
    #firstHalf = driver.current_url.index('currentJobId=')+len('currentJobId=')
    firstHalf = driver.current_url.index('?')
    secondHalf = driver.current_url.index('keywords')
#     jobId = temp.find('a',href=True)['href']
#     jobId= jobId.split('/')[3]
    newUrl = currUrl[:firstHalf] + '?currentJobId='+id+ '&'+ currUrl[secondHalf:]
    job_links.append(newUrl)

In [189]:
print(job_links)
print(len(job_links))

['https://www.linkedin.com/jobs/search/?currentJobId=2473948505&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2486288184&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2477257225&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2473932077&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2469597134&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2505171031&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2486211179&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2484382977&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2487428041&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2483648907&keywords=software%20engineer', 'https://www.linkedin.com/jobs/search/?currentJobId=2482229

In [190]:
browser.get(job_links[0])

In [65]:
# setting up list for job information
#https://amandeepsaluja.com/extracting-job-information-from-linkedin-jobs-using-beautifulsoup-and-selenium/
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
   # job_titles = job.find("span", class_="screen-reader-text").text
    #post_title.append(job_titles)
    
    # linkedin job id
   # print(job.find('a', href=True)['href'].split('/'))
    try:
        id = job.find('a', href=True)['href'].split('/')[3]
    except:
        print(job.find('a', href=True)['href'])
    #job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(id)
    
    # company name
#     company_names = job.select_one('img')['alt']
#     company_name.append(company_names)
    
    # job location
   # job_locations = job.find("span", class_="job-result-card__location").text
    #job_location.append(job_locations)
    
    # posting date
  #  post_dates = job.select_one('time')['datetime']
 #   post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

TypeError: 'NoneType' object is not subscriptable

In [66]:
job_id

['2473948505',
 '2486288184',
 '2479908835',
 '2473932077',
 '2469597134',
 '2505171031',
 '2486211179',
 '2484382977']

In [121]:
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
#     job_titles = job.find("span", class_="screen-reader-text").text
#     post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    print(job_ids)
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
#     job_locations = job.find("span", class_="job-result-card__location").text
#     job_location.append(job_locations)
    
    # posting date
#     post_dates = job.select_one('time')['datetime']
#     post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

/jobs/view/2479707838/?eBP=CwEAAAF4zTBe81BidYOyNjl6hmBTCajakrqhVZYSjgSNhswOONYh-2bv-zHIk1SU5c0P6F78xxpjGW22MyvbHtFnmNwrseT_EIed2B8q2U0KDahJA1jiZ59ICK5ewdSBsNVH8M-jkfFLrryxqC6Mj5HvtagFpJjd97N5epngo2ukbtZM-8G11CTVXr7gLq6AkSj_IyrXmwnFVm6etiU8ZH59VUxpeNtDm40_DLPd-mkfOc0C8DAj6kdZ-R-lM4qS8e9vuNhkA8CU4QhVmLcgtGTOttCFOPFfalqQgX4w16XePdOlcEV3ABZBVAnNbiXYZ-USS2_32jKCtiMgPJrMTZIBKTdE7cnG6OA4&recommendedFlavor=ACTIVELY_HIRING_COMPANY&refId=uD3mxmBSoN043b40kT6%2FMQ%3D%3D&trackingId=if1Ou%2FfcL9HRT%2FeC4LxEyw%3D%3D&trk=flagship3_search_srp_jobs
/jobs/view/2473948505/?eBP=CwEAAAF4zTBe8-s1XUAhzZCQPd84OF_LIeE8IBkD-71wXRbvBaHR6ebjwh_4CyF--8hoB99Kc1nnRhVRT4iifXxdWccCc5C9Ym1CnKMld1Ep_prO1x6ls4zH7bv4OjPpIenFzYg-LpT4IIC4pTZHjkw9JVSj2cb_h7JoOg4ACxUrbE-oA4IjAcc6wJ7lXMa-Vq0H6KGhThpR8bv5BETP8B616JriqXq3LxSxwBhQg6r9lcQsWNNm8wtVLkvrTEqeLIlVzJlmTeCYKVOS8RdeNQ1Ei1Tcm-Q4AA3nT7TymFBuaG6znpvzloT74Kdiq2_cr1DUmgTfJa5E1L54OWqa7-OaKVmGdKSt5arbH-qJetmHsh4CPTAP3z9a11vwsu8apd3A3SpDDSQt&recommendedFlavor=ACTIVELY_HIRING_COMPA

TypeError: 'NoneType' object is not subscriptable

In [117]:
len(job_id)

12

In [ ]:
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))